In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error


## PREPARAR DATOS

In [15]:
va22 = pd.read_csv("df_22.csv")
va23 = pd.read_csv("df_23.csv")
va24 = pd.read_csv("df_24.csv")
data = pd.concat([va22, va23, va24], ignore_index=True)

# Asegurarse de que la columna de temperatura se llama 'temperatura'
data['tmed'].values

array([19.8, 10.2, 13.6, ...,  nan, 17. , 16.5])

In [16]:
data.dropna()

,fecha,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,indicativo_id,nombre_id,provincia_id
0,2022-01-01,115,19.8,0.0,13.4,06:00:00,26.1,13:10:00,7.0,1.9,4.4,13:40:00,41.0,95.0,22:10:00,16.0,11:20:00,1,1,1
2,2022-01-01,410,13.6,0.0,5.7,07:00:00,21.5,14:20:00,18.0,1.9,9.4,11:50:00,59.0,83.0,06:50:00,29.0,14:20:00,3,3,3
3,2022-01-01,808,14.8,0.0,6.8,07:00:00,22.8,14:10:00,33.0,1.4,5.0,15:10:00,48.0,69.0,23:40:00,25.0,13:50:00,4,4,4
4,2022-01-01,582,6.4,0.0,-2.0,08:00:00,14.9,15:00:00,22.0,0.8,2.2,03:40:00,70.0,96.0,08:40:00,41.0,15:00:00,5,5,5
5,2022-01-01,17,12.0,0.0,9.5,01:40:00,14.5,13:00:00,26.0,3.1,9.7,16:40:00,91.0,99.0,00:00:00,83.0,14:50:00,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908528,2024-11-01,790,14.5,0.0,10.5,23:59:00,18.5,13:50:00,12.0,1.4,7.2,02:10:00,70.0,84.0,07:30:00,53.0,13:50:00,199,198,24
908535,2024-11-01,87,18.6,3.0,16.8,07:00:00,20.5,11:50:00,10.0,1.7,5.6,08:30:00,91.0,100.0,00:30:00,73.0,14:00:00,228,226,51
908536,2024-11-01,701,15.0,0.0,10.2,23:58:00,19.8,13:38:00,12.0,3.1,8.6,15:10:00,81.0,98.0,23:59:00,57.0,13:40:00,525,521,8
908537,2024-11-01,2369,10.5,0.0,6.5,00:30:00,14.5,15:00:00,32.0,8.1,13.1,17:10:00,20.0,41.0,16:40:00,7.0,00:00:00,237,234,1


### Modelo RNN

In [20]:
# Normalización de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
temperatures = scaler.fit_transform(temperatures.reshape(-1, 1))

# Crear secuencias de datos
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 10
X, y = create_dataset(temperatures, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(SimpleRNN(25, return_sequences=True, input_shape=(X_train.shape[1], 1)))  
model.add(SimpleRNN(25))  
model.add(Dense(1))  # Capa de salida para la predicción de temperatura

# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Configurar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-6)

# Entrenamiento del modelo 
model.fit(X_train, y_train, epochs=100, batch_size=64, callbacks=[early_stopping,reduce_lr])

# Hacer predicciones
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)



NameError: name 'temperatures' is not defined

In [22]:
# Calcular métricas
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')

# Visualización de resultados
plt.figure(figsize=(14, 5))
plt.plot(y_test, label='Valores Reales', color='blue')
plt.plot(predictions, label='Predicciones', color='red')
plt.title('Predicciones de Temperatura')
plt.xlabel('Tiempo')
plt.ylabel('Temperatura')
plt.legend()
plt.show()

NameError: name 'y_test' is not defined

In [24]:
plt.figure(figsize=(14, 5))
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento', color='orange')
plt.plot(history.history['val_loss'], label='Pérdida de Validación', color='green')
plt.title('Pérdida durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

NameError: name 'history' is not defined

<Figure size 1400x500 with 0 Axes>

## Modelo LSTM

In [ ]:
modelo_lstm = Sequential()
modelo_lstm.add(LSTM(25, input_shape=(X_train.shape[1], 1)))  
modelo_lstm.add(Dense(1))  
modelo_lstm.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-6)

# Entrenar el modelo y guardar el historial
historial = modelo_lstm.fit(X_train, y_train, epochs=50, batch_size=64, callbacks=[early_stopping, reduce_lr])

# Visualizar las métricas
plt.figure(figsize=(12, 6))
plt.plot(historial.history['loss'], label='Pérdida de Entrenamiento')
plt.title('Pérdida del Modelo LSTM a lo Largo de las Épocas')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid()
plt.show()

# Evaluar el modelo
loss = modelo_lstm.evaluate(X, y)
print(f'Pérdida del modelo: {loss}')

# Obtener las predicciones
predicciones = modelo_lstm.predict(X)

# Calcular métricas adicionales
rmse = mean_squared_error(y, predicciones, squared=False)
r2 = r2_score(y, predicciones)

print(f'RMSE: {rmse}')
print(f'R²: {r2}')

plt.figure(figsize=(10, 6))
plt.plot(y, label='Valores reales')
plt.plot(predicciones, label='Predicciones')
plt.title('Comparación de Valores Reales y Predicciones')
plt.xlabel('Muestras')
plt.ylabel('Valores')
plt.legend()
plt.show()